# Additional Examples

In [1]:
using RHEOS

[PyPlot](https://github.com/JuliaPy/PyPlot.jl) needs to be installed to run these examples and display plots of the data.

In [2]:
using PyPlot

[ Info: Installing matplotlib via the Conda matplotlib package...
[ Info: Running `conda install -q -y matplotlib` in root environment
Solving environment: ...working... done

## Package Plan ##

  environment location: /home/runner/.julia/conda/3

  added / updated specs:
    - matplotlib


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    brotli-1.0.9               |       he6710b0_2         375 KB
    cycler-0.10.0              |   py39h06a4308_0          16 KB
    dbus-1.13.18               |       hb2f20db_0         504 KB
    expat-2.4.1                |       h2531618_2         168 KB
    fontconfig-2.13.1          |       h6c09931_0         250 KB
    fonttools-4.25.0           |     pyhd3eb1b0_0         632 KB
    freetype-2.10.4            |       h5ab3b9f_0         596 KB
    glib-2.69.0                |       h5202010_0         1.7 MB
    gst-plugins-base-1.14.0    |       h821

## Example 1

- Loading experimental data from a .csv file
- Fitting a model
- Compare original data with fitted model prediction

In [3]:
# Make sure the examples folder is the current directory
# check by typing "pwd()"

# Import data
data = importcsv("assets/example1_data.csv", t_col = 1, ϵ_col = 2, σ_col = 3)

# Plot data
fig, ax = subplots(1, 1, figsize = (7, 5))
ax.plot(data.t, data.σ, ".", color = "green")
ax.plot(data.t, data.ϵ, "-", color = "blue")
ax.set_ylabel("Strain (blue), Stress (green)")
ax.set_xlabel("Time")

PyObject Text(0.5, 0, 'Time')

In [4]:
# We now fit a Maxwell model
maxwell_model = modelfit(data, Maxwell, strain_imposed)

┌ Warning: Initial values for model parameters are set to [0.5, 0.5] by default
└ @ RHEOS ~/work/RHEOS.jl/RHEOS.jl/src/processing.jl:261
Time: 4.200163983 s, Why: XTOL_REACHED, Parameters: [4.0054017305374146, 2.003000020980835], Error: 0.022310985023644012



Model name: maxwell

Free parameters: 


Fixed parameters: (η = 4.0054017305374146, k = 2.003000020980835)

                ___
            _____| |________╱╲  ╱╲  ╱╲  ___
                _|_|          ╲╱  ╲╱  ╲╱
                  η                  k
               

Note that the fitting function requires guidance regarding the type of testing used. It helps optimise the fitting process.

The data in this example is the stress response to a strain ramp followed by plateau. It therefore corresponds to a strain imposed process.

We now want to calculate the stress values predicted by the model given the experimental strain data. Let's create a new data set with the strain profile.

In [5]:
maxwell_predict = extract(data, strain_only)
# and calculate the stress based on the model
maxwell_predict = modelpredict(maxwell_predict, maxwell_model)
# Now we can plot data and model together for comparison

# Plot data
fig, ax = subplots(1, 1, figsize = (7, 5))
ax.plot(data.t, data.σ, ".", color = "green")
ax.plot(maxwell_predict.t, maxwell_predict.σ, color = "red")
ax.set_xlabel("Time")
ax.set_ylabel("Stress")

PyObject Text(0, 0.5, 'Stress')

## Example 2

This script is a slight modification of Example 1 to present to the user the possibility of creating new `RheoModelClass` from an existing one with some of the parameters frozen to specific values. As an example, we fix the spring constant of the model above (k) to 2 and we let RHEOS fit the viscosity η.

In [6]:
Maxwell_springFix = freeze_params(Maxwell, k = 2)

freeze_params is deprecated - use freezeparams instead.



Model name: maxwell

Free parameters: η


Fixed parameters: (k = 2.0,)

                ___
            _____| |________╱╲  ╱╲  ╱╲  ___
                _|_|          ╲╱  ╲╱  ╲╱
                  η                  k
               

In [7]:
maxwellD_model = modelfit(data, Maxwell_springFix, strain_imposed)
maxwellD_predict = extract(data, strain_only)
# and calculate the stress based on the model
maxwellD_predict = modelpredict(maxwellD_predict, maxwellD_model)
# Now we can plot data and model together for comparison

# Plot data
fig, ax = subplots(1, 1, figsize = (7, 5))
ax.plot(data.t, data.σ, ".", color = "green")
ax.plot(maxwellD_predict.t, maxwellD_predict.σ, color = "red")
ax.set_xlabel("Time")
ax.set_ylabel("Stress")

┌ Warning: Initial values for model parameters are set to [0.5] by default
└ @ RHEOS ~/work/RHEOS.jl/RHEOS.jl/src/processing.jl:261
Time: 0.005050123 s, Why: XTOL_REACHED, Parameters: [4.00555419921875], Error: 0.022412669261910354


PyObject Text(0, 0.5, 'Stress')

## Example 3

This script shows how to use RHEOS to explore the behaviour of various models This involves:
- Creating a strain function
- Defining models based on parameter values
PyPlot needs to be installed to run these examples and display plots of the data.

In [8]:
# Creates a time only dataset
dϵ = timeline(t_end = 10)
# calculates strain data by applying a function of time
dϵ = strainfunction(dϵ, t -> sin(t))

# Plot strain data
fig, ax = subplots(1, 1, figsize = (7, 5))
ax.plot(dϵ.t, dϵ.ϵ, "--b")

# we can now simulate various models based on this strain only dataset
# Let's study the role of the dashpot strength in the MAxwell model
for η in [0.1, 0.3, 1, 3, 10]
    maxwell_model = RheoModel(Maxwell, k = 2., η = η)
    d_maxwell = modelpredict(dϵ, maxwell_model)
    ax.plot(d_maxwell.t, d_maxwell.σ)
end
ax.set_xlabel("Time")
ax.set_ylabel("Stress")
ax.grid("on")

┌ Warning: Assignment to `maxwell_model` in soft scope is ambiguous because a global variable by the same name exists: `maxwell_model` will be treated as a new local. Disambiguate by using `local maxwell_model` to suppress this warning or `global maxwell_model` to assign to the existing global variable.
└ @ string:13


---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*